### Note to the Readers:

When I wrote the codes for the first half of this excercise, I didn't know much about the "Groupby" function and its capabilities. So the way I did in the first half was not really an efficient way to do it as it feels like I'm reinventing the wheel for what << sum(), mean(), count() >> can do. But I do hope you enjoy my logic as you step through the codes I developed to get the results.


In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load
file_to_load = "Resources/game_purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load, low_memory=False)





## Player Count

* Display the total number of players


In [2]:
# clean the data and remove all duplicates in player's names
purchase_data_uniqueSN = purchase_data[['SN', 'Gender']].drop_duplicates()

# count how many players there are
total_players = len(purchase_data_uniqueSN['SN'])

# create a dataframe for the total player to show a nice table format
tp_df = pd.DataFrame({"Total Players" : [total_players]})
tp_df

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [3]:
# find the total number of unique items
unique_item = purchase_data['Item ID'].unique()
num_unique_item = len(unique_item)

# find the total purchases
total_num_purchase = purchase_data['Item ID'].count()

# average price and total price
ave_price = round(purchase_data['Price'].mean(), 2)
total_rev = round(purchase_data['Price'].sum(), 2)


# create dataframe for summary table
sum_tbl_data = {"Number of Unique Items" : num_unique_item,"Average Price" : ave_price,
"Number of Purchases" : total_num_purchase, "Total Revenue" : total_rev}
sum_tbl_df = pd.DataFrame(sum_tbl_data, index=[0])

# format the summary dataframe
sum_tbl_df['Average Price'] = sum_tbl_df['Average Price'].map("${:.2f}".format)
sum_tbl_df['Total Revenue'] = sum_tbl_df['Total Revenue'].map("${:,.2f}".format)
sum_tbl_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [4]:
# count total "female" and "male" purchase
unique_SN_gender_count = purchase_data_uniqueSN["Gender"].value_counts()

# add the value counts into dataframe to show nice table
unique_SN_gender_df = pd.DataFrame(unique_SN_gender_count)

# calculate percentage of genders and format the numbers to show " % " symbol
unique_SN_gender_df["Percentage of Players"] = round(unique_SN_gender_df["Gender"] / total_players * 100, 2)
unique_SN_gender_df["Percentage of Players"] = unique_SN_gender_df["Percentage of Players"].map('{:.2f}%'.format)

# Rename the column to show total counts from the default name "Gender" produced by the value_counts() function
unique_SN_gender_df.rename(columns={"Gender" : "Total Counts"})

,Total Counts,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [5]:
# Group all duplicates by SN names and all Genders
uni_dir = purchase_data.groupby(['SN','Gender'])['Price'].unique()
uni_df = pd.DataFrame(uni_dir)
uni_df.reset_index(inplace=True)

# my idea in this section is to create a list to store all purchase prices for each player name
#create a column (series) in dataframe to store "price" data for each player
uni_df["Sum per individual"] = ""


# Looping to find the total spending of each female
for each_purchase in range(len(uni_df["SN"])):
    uni_df["Sum per individual"][each_purchase] = uni_df["Price"][each_purchase].sum()



In [6]:
#===================== FOR FEMALE =======================

# Using loc function to filter out only female purchases
# SN names have duplicates
female_pur_data = purchase_data.loc[purchase_data["Gender"] == "Female", :]

# Total female purchase count and cost, including SN duplicates
total_female_purchase_count = len(female_pur_data["Gender"])
total_female_purchase_cost = female_pur_data["Price"].sum()


# Eliminate all SN duplicates for Female Category
uni_female_df = uni_df.loc[uni_df['Gender'] == "Female", :]
total_femaleSN_unique = len(uni_female_df)

# Calculate average purchase price
ave_pur_price_female = total_female_purchase_cost / total_female_purchase_count

# Calculate Avg Total Purchase per Persontotal_femaleSN_unique
avgtotal_purchase_per_female = total_female_purchase_cost / total_femaleSN_unique


In [7]:
#===================== FOR MALE =======================

# Using loc function to filter out only male purchases
# SN names have duplicates
male_pur_data = purchase_data.loc[purchase_data["Gender"] == "Male", :]

# Total male purchase count and cost, including SN duplicates
total_male_purchase_count = len(male_pur_data["Gender"])
total_male_purchase_cost = male_pur_data["Price"].sum()


# Eliminate all SN duplicates for male Category
uni_male_df = uni_df.loc[uni_df['Gender'] == "Male", :]
total_maleSN_unique = len(uni_male_df)

# Calculate average purchase price
ave_pur_price_male = total_male_purchase_cost / total_male_purchase_count


# Calculate Avg Total Purchase per Person
avgtotal_purchase_per_male =  total_male_purchase_cost / total_maleSN_unique


In [8]:
#===================== FOR OTHER / NON-DISCLOSE (O-ND) =======================

# Using loc function to filter out only O-ND purchases
# SN names have duplicates
nd_pur_data = purchase_data.loc[purchase_data["Gender"] == "Other / Non-Disclosed", :]


# Total O-ND purchase count and cost, including SN duplicates
total_nd_purchase_count = len(nd_pur_data["Gender"])
total_nd_purchase_cost = nd_pur_data["Price"].sum()


# Eliminate all SN duplicates for O-ND Category
uni_nd_df = uni_df.loc[uni_df['Gender'] == "Other / Non-Disclosed", :]
total_ndSN_unique = len(uni_nd_df)

# Calculate average purchase price
ave_pur_price_nd = total_nd_purchase_cost / total_nd_purchase_count


# Calculate Avg Total Purchase per Person
avgtotal_purchase_per_nd = total_nd_purchase_cost / total_ndSN_unique


In [9]:
# Build summary data table with column names, data is imported from each gender category
sum_table = pd.DataFrame(
    {"Gender" : ["Female", "Male", "Other / Non-Disclosed"],
     "Purchase Count" : [total_female_purchase_count, total_male_purchase_count, total_nd_purchase_count],
     "Average Purchase Price" : [ave_pur_price_female, ave_pur_price_male, ave_pur_price_nd],
     "Total Purchase Value" : [total_female_purchase_cost, total_male_purchase_cost, total_nd_purchase_cost],
     "Avg Total Purchase per Person" : [avgtotal_purchase_per_female, avgtotal_purchase_per_male, avgtotal_purchase_per_nd]       
    })


# Use Map to re-format all price columns
sum_table["Total Purchase Value"] = sum_table["Total Purchase Value"].map("${:,.2f}".format)
sum_table["Average Purchase Price"] = sum_table["Average Purchase Price"].map("${:.2f}".format)
sum_table["Avg Total Purchase per Person"] = sum_table["Avg Total Purchase per Person"].map("${:.2f}".format)
sum_table.head()


,Gender,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
0,Female,113,$3.20,$361.94,$4.47
1,Male,652,$3.02,"$1,967.64",$4.07
2,Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [10]:
# create bin and labels for Age Ranges
bin_label = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']
bin_age = [0, 9, 14, 19, 24, 29, 34, 39, purchase_data["Age"].max() + 10]

# Create table with unique SN names
age_dir = purchase_data.groupby(['SN','Age'])['Price'].unique()
age_df = pd.DataFrame(age_dir)
age_df.reset_index(inplace=True)



In [11]:
# =================== Display Age Demographics Table ===================

# binning the dataframe with Age Ranges
age_df["Age Ranges"] = pd.cut(age_df["Age"], bin_age, labels=bin_label) 

# count the 'Age Ranges' data points
age_table = age_df["Age Ranges"].value_counts()
age_table_df = pd.DataFrame(age_table)

# calculate the Percentage of Players
age_table_df["Percentage of Players"] = round(age_table / age_table.sum()*100, 2).map("{:.2f}%".format)

# rename the column names of the dataframe
age_table_df.columns = ["Total Count", "Percentage of Players"]

# display the dataframe and sort the 'Age Ranges' small to large
age_table_df
age_table_df.sort_index()

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [12]:
# bin and add "Age Ranges" into the original dataframe
purchase_data["Age Ranges"] = pd.cut(purchase_data["Age"], bin_age, labels=bin_label)

In [13]:
# create the groupby dataframe based on Age Ranges
grp_age = purchase_data.groupby('Age Ranges')

In [14]:
# Create the entire dataframe from series of interest
pur_ana_age = pd.DataFrame({
    "Purchase Count" : grp_age['Price'].count(), # series of total purchase count
    "Average Purchase Price" : grp_age['Price'].mean(), # series of average purchase price
    "Total Purchase Price" : grp_age['Price'].sum(), # series of total purchase price
    "Avg Total Purchase per Person" :  grp_age['Price'].sum()/age_df["Age Ranges"].value_counts() # series of avr purchase per person
})

# reformat all the columns that have monetary values, add thousands seperator into total purchase price
pur_ana_age["Average Purchase Price"] = pur_ana_age["Average Purchase Price"].map("${:.2f}".format)
pur_ana_age["Total Purchase Price"] = pur_ana_age["Total Purchase Price"].map("${:,.2f}".format)
pur_ana_age["Avg Total Purchase per Person"] = pur_ana_age["Avg Total Purchase per Person"].map("${:.2f}".format)

# Display the summary data frame
pur_ana_age

,Purchase Count,Average Purchase Price,Total Purchase Price,Avg Total Purchase per Person
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [15]:
print(pur_ana_age["Avg Total Purchase per Person"].min())

$3.19


##  Top Spenders

 
* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame

In [16]:
# Group all the purchase by SN 
grp_sn = purchase_data.groupby('SN')

# Run basic calculations to obtain the results 
grp_sn_count = grp_sn['SN'].count()
grp_sn_sum = grp_sn['Price'].sum()
grp_sn_mean = grp_sn['Price'].mean()


In [17]:
# Create a summary data frame to hold the results
sum_top_spender = pd.DataFrame({
    "Purchase Count" : grp_sn_count,
    "Average Purchase Price" : grp_sn_mean,
    "Total Purchase Value" : grp_sn_sum
})

# Sort the total purchase value column in descending order
sorted_sum_top_spender = sum_top_spender.sort_values("Total Purchase Value", ascending=False)

# format dataframe to show " $ "
sorted_sum_top_spender["Average Purchase Price"] = sorted_sum_top_spender["Average Purchase Price"].map("${:.2f}".format)
sorted_sum_top_spender["Total Purchase Value"] = sorted_sum_top_spender["Total Purchase Value"].map("${:,.2f}".format)

# display the summary dataframe
sorted_sum_top_spender.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


##  Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [18]:
# Retrieve the Item ID, Item Name, and Item Price columns
filtered_purchase_data = purchase_data[["Item ID", "Item Name", "Price"]]

# Group by Item ID and Item Name
grp_item_list = filtered_purchase_data.groupby(["Item ID", "Item Name"])

# Perform calculations to obtain purchase count, item price, and total purchase value
# Create a summary data frame to hold the results
grp_item_sum_df = pd.DataFrame({
    "Purchase Count" : grp_item_list['Item Name'].count(),
    "Item Price" : grp_item_list['Price'].mean(),
    "Total Purchase Value" : grp_item_list['Price'].sum()
})

# Make a copy to manipulate the table format so I won't affect any operations later that needs the original dataframe again
copy_grp_item_sum_df = grp_item_sum_df
copy_grp_item_sum_df

# Sort the purchase count column in descending order
sorted_grp_item_sum_df_purchase_count = copy_grp_item_sum_df.sort_values(['Purchase Count'], ascending=False)

# Give the displayed data cleaner formatting
sorted_grp_item_sum_df_purchase_count['Item Price'] = sorted_grp_item_sum_df_purchase_count['Item Price'].map('${:,.2f}'.format)
sorted_grp_item_sum_df_purchase_count['Total Purchase Value'] = sorted_grp_item_sum_df_purchase_count['Total Purchase Value'].map('${:,.2f}'.format)

# Display a preview of the summary data frame
sorted_grp_item_sum_df_purchase_count

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16
...,...,...,...,...
104,Gladiator's Glaive,1,$1.93,$1.93
23,Crucifer,1,$1.99,$1.99
180,Stormcaller,1,$3.36,$3.36


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [19]:
# Make a copy to manipulate the table format so I won't affect any operations later that needs the original dataframe again
most_profitable_df = grp_item_sum_df

# sort the dataframe by total purchase value in descending order
sorted_most_profitable_df = most_profitable_df.sort_values(['Total Purchase Value'], ascending=False)

# reformat the dataframe to show " $ " and the thousand seperator
sorted_most_profitable_df['Item Price'] = sorted_most_profitable_df['Item Price'].map('${:,.2f}'.format)
sorted_most_profitable_df['Total Purchase Value'] = sorted_most_profitable_df['Total Purchase Value'].map('${:,.2f}'.format)

# show dataframe
sorted_most_profitable_df

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
...,...,...,...,...
125,Whistling Mithril Warblade,2,$1.00,$2.00
126,Exiled Mithril Longsword,1,$2.00,$2.00
23,Crucifer,1,$1.99,$1.99


In [20]:
# I'm curious to know all info about the top spender for my conclusion
locate_top_spender = purchase_data.loc[purchase_data["SN"] == "Lisosia93", :]
locate_top_spender

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Ranges
74,74,Lisosia93,25,Male,89,"Blazefury, Protector of Delusions",4.64,25-29
120,120,Lisosia93,25,Male,24,Warped Fetish,3.81,25-29
224,224,Lisosia93,25,Male,157,"Spada, Etcher of Hatred",4.80,25-29
603,603,Lisosia93,25,Male,141,Persuasion,3.19,25-29
609,609,Lisosia93,25,Male,40,Second Chance,2.52,25-29


## Conclusion based on the Data Analysis

1. Players in the age range of "20-24" occupy 44.79% of the data population, which is the most popular "age rage" among all players.
2. Majority of the players are "male" with approx. 84% of the data population.
3. The most popular item is also the most profitable item, which is "Oathbreaker, Last Hope of the Breaking Storm".
4. "Lisosia93"  is the top spender, gender is male, 25 years old, and belong to age group of "25-29", and spent total 18.96 USD in five total purchases.
5. Age Range of "40+" has the lowest "Avg Total Purchase per Person", with only "$3.19 / person"